In [1]:
using LinearAlgebra
using Plots

In [13]:
function get_nodes_ix(nodes, nelem)
    ix = zeros(nodes)
    for i in 1:nodes
        ix[i] = (nodes - 1) * (nelem - 1) + i
    end
    return map(y -> round(Int,y), ix)  # to avoid float indices.
end

get_nodes_ix (generic function with 1 method)

In [14]:
function simpson_rule_l(i, j)
    
    ϕ_diff_0 = x -> -1
    ϕ_diff_1 = x -> 1
    elem_diff = [ϕ_diff_0 ϕ_diff_1]
    
    q = [0 1/2 1]
    w = [1/6 4/6 1/6]
    rv = 0
    for k in 1:3
        rv = rv + elem_diff[i](q[k]) * elem_diff[j](q[k]) * w[k]
    end

    return rv
end

simpson_rule_l (generic function with 1 method)

In [15]:
function simpson_rule_f(i, j, fun, h)
    q = [0 1/2 1]
    return fun(i*h + q[j]*h) * h
end

simpson_rule_f (generic function with 2 methods)

In [16]:
function ith_rhs(i, fun, h)
    rv = 0
    for j in 1:3
        rv = rv + simpson_rule_f(i, j, fun, h)
    end
    return rv
end

ith_rhs (generic function with 2 methods)

In [22]:
function finite_elements(nelem, fun)
    h = 1/nelem
    f = x -> 1
    F = zeros(nelem + 1)
    # local matrix
    nodes_by_element = 2
    local_matrix = zeros(nodes_by_element, nodes_by_element)
    for i in 1:nodes_by_element, j in 1:nodes_by_element
        local_matrix[i, j] = simpson_rule_l(i, j)
    end

    # global matrix
    global_matrix = zeros(nelem+1, nelem+1)
    for i in 1:nelem
        nodes_ix = get_nodes_ix(nodes_by_element, i)
        i1, i2 = nodes_ix[1], nodes_ix[2]
        global_matrix[i1:i2, i1:i2] = global_matrix[i1:i2, i1:i2] + local_matrix

        fi = ith_rhs(i, f, h)
        F[i] = F[i] .+ fi
    end
    F[1], F[end] = 0, 0
    global_matrix = global_matrix .* h
    
    # border conditions
    global_matrix[2:end, 1] .= 0
    global_matrix[1, 2:end] .= 0
    global_matrix[1, 1] = 1
    
    global_matrix[1:end-1, end] .= 0
    global_matrix[end, 1:end-1] .= 0
    global_matrix[end, end] = 1
    return global_matrix
end

finite_elements (generic function with 1 method)

In [27]:
finite_elements(4, 0)

[0.0, 0.75, 0.75, 0.75, 0.0]

5×5 Matrix{Float64}:
 1.0   0.0    0.0    0.0   0.0
 0.0   0.5   -0.25   0.0   0.0
 0.0  -0.25   0.5   -0.25  0.0
 0.0   0.0   -0.25   0.5   0.0
 0.0   0.0    0.0    0.0   1.0